In [58]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

In [59]:
hc = 12.39841875 #keV*Angström
d_si = 3.1358

In [60]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen/'

In [61]:
var_pc = pc_henrik

In [62]:
cd $var_pc'Messdaten/'

/home/arrow13/Git/FP/Roentgen/Messdaten


# Implementirung der Funktionen für die Auswertung

## Die Peak sollen mit der Gaussverteilung angefitten werde, diese wird hier implementier, wie ein wrappen für automatische fits

# Gauss
<h1>$f(x) = \frac{a}{\sqrt{2 \cdot pi \cdot \sigma^2}} \cdot e^{- \frac{{(x- \sigma )}^2}{2 \cdot \sigma^2}} $ </h1>

In [ ]:
def gauss(x,a,mu,sigma):
    return (a/(np.sqrt(2*3.14*sigma**2))*np.exp(-((x-mu)**2)/(2.*sigma**2)))

# ${\chi_{red}}^2$ / Goodness of a fit
<h1> ${\chi_{red}}^2 = \frac{1}{\nu}\sum\frac{{(O-E)}^2}{\sigma^2}$ 

In [ ]:
def red_chi_gauss(data, model, deg):
    chisq = np.sum(np.square(data-model)/np.power(data,0.5))
    nu = data.size-1-deg    # -4: -1 for the equation and -3 from the degrees of freedom
    if nu is 0:
        return "err: nu = 0"
    return chisq/nu

In [ ]:
def fit_gauss(x,data,guess=[1,1,1]):
    '''
        returns the parameter from the fit, the standard deviation and the reduced chisquare
    '''
    para, cov = opt.curve_fit(gauss,x,np.sqrt(data),guess,maxfev=10000)
    red_chi = red_chi_gauss(data,gauss(x,para[0],para[1],para[2]),3)
    return para, np.sqrt(np.diag(cov)), red_chi

# Bragg-reflexion
$n \cdot \lambda = 2 \cdot d \cdot sin(\theta)$

# Energie
$E = \frac{h \cdot c}{2 \cdot d \cdot sin(\theta)}$

In [ ]:
def energie(phi,delta_phi):
    theta = phi/2
    e = hc/(2*d_si*np.sin(theta*(3.14/180)))
    delta_e = (e*delta_phi/2)/np.tan(theta*(3.14/180))
    return e, delta_e

In [ ]:
def intensität(a,delta_a,sigma,delta_sigma):
    i = a/(np.sqrt(2*3.14*sigma**2))
    delta_i = np.sqrt((delta_a/np.sqrt(2*3.14*sigma**2))**2 + ((a*delta_sigma)/(np.sqrt(2*3.14*sigma**4)))**2)
    return i, delta_i

In [ ]:
def d(e,delta_e,phi,delta_phi,n):
    d = (n*hc)/(2*e*np.sin((phi/2)*(3.14/180)))
    delta_d = d*np.sqrt(((delta_phi/2)/np.tan((phi/2)*(3.14/180)))**2+(delta_e/e)**2)

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = ??, A$_1$ = ??
+ U$_2$ = ??, A$_2$ = ??
+ U$_3$ = ??, A$_3$ = ??

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdefraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

# Ersterwert: Spannung
# Zweiterwert: Anodenstrom


# Vorbereitung für den Voigt-Fit

In [ ]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
mulit_voigt_1 = voigt_1 + voigt_2

In [ ]:
data = np.loadtxt("Si111_40_30.xy").T
plt.plot(data[0],data[1])
plt.show()

In [ ]:
params_1 = voigt_1.guess(data[1],x=data[0])
params_2 = voigt_2.guess(data[1],x=data[0])
params_3 = voigt_3.guess(data[1],x=data[0])
params_mulit = mulit_voigt_1.make_params()

params_mulit['voigt_1center'].set(28.43)
params_mulit['voigt_1center'].set(28.50)
params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [ ]:
np.arange(np.where(data[0] > 28.2)[0][0], np.where(data[0] < 28.7)[0][0])

In [ ]:
np.where(data[0] < 28.7)

In [ ]:
fit_1 = voigt_1.fit(data[1],params_1,x=data[0])
fit_2 = voigt_2.fit(data[1],params_2,x=data[0])
fit_3 = voigt_3.fit(data[1],params_3,x=data[0])
fit_mulit = mulit_voigt_1.fit(data[1],params_mulit,x=data[0])
print(fit_mulit.fit_report())

In [ ]:
peak_1_range = np.arange(np.where( 25.5 < data[0])[0][0],np.where( data[0] > 25.8)[0][0])
peak_2_range = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
peak_3_range = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])

In [ ]:
params_1 = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])
params_2 = voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range])
params_3 = voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range])

params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [ ]:
fit_1 = voigt_1.fit(data[1][peak_1_range],params_1,x=data[0][peak_1_range],weights=1/np.sqrt(data[1][peak_1_range]))
fit_2 = voigt_2.fit(data[1][peak_2_range],params_2,x=data[0][peak_2_range],weights=1/np.sqrt(data[1][peak_2_range]))
fit_3 = voigt_3.fit(data[1][peak_3_range],params_3,x=data[0][peak_3_range],weights=1/np.sqrt(data[1][peak_3_range]))

In [ ]:
fit_1.plot()
fit_2.plot()
fit_3.plot()
plt.show()

In [ ]:
print(fit_1.fit_report())
print(fit_2.fit_report())
print(fit_3.fit_report())

In [ ]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.plot(data[0][peak_1_range],fit_1.best_fit)
plt.plot(data[0][peak_2_range],fit_2.best_fit)
plt.plot(data[0][peak_3_range],fit_3.best_fit)
plt.show()

In [ ]:
multivoigt = voigt_1 + voigt_2 + voigt_3

In [ ]:
params = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])

In [ ]:
params.update(voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range]))

In [ ]:
params.update(voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range]))

In [ ]:
err = np.sqrt(data[1])
err = np.where(err==0, 1, err)

In [ ]:
fit_4 = multivoigt.fit(data[1],params,x=data[0],weights=1/err)

In [ ]:
fit_4.best_values

In [ ]:
print(fit_4.fit_report())

In [ ]:
fit_4.plot()
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(data[0],fit_4.best_fit,'r',label='Multi-Voigt-Fit')
plt.errorbar(data[0],data[1],np.sqrt(data[1]),label='Messdaten')
plt.xlim([25,29])
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

# Pulverdiffraktometrie Auswertung

In [124]:
data_pulver = np.loadtxt("Pulver_Ni.xy").T

In [125]:
data_pulver

array([[  10.00417782,   10.01253345,   10.02088909, ...,  142.58303095,
         142.59138659,  142.59974222],
       [   9.        ,   17.        ,   16.        , ...,    1.        ,
           1.        ,    0.        ]])

In [126]:
len(data_pulver[0])

15870

In [127]:
len(data_pulver[1])

15870

In [ ]:
plt.plot(data_pulver[0],data_pulver[1], '.')
plt.show()

In [128]:
voigt_peak_1 = models.VoigtModel(prefix='voigt_peak_1')
voigt_peak_2 = models.VoigtModel(prefix='voigt_peak_2')
voigt_peak_3 = models.VoigtModel(prefix='voigt_peak_3')
voigt_peak_4 = models.VoigtModel(prefix='voigt_peak_4')
voigt_peak_5 = models.VoigtModel(prefix='voigt_peak_5')
voigt_peak_6 = models.VoigtModel(prefix='voigt_peak_6')
voigt_peak_7 = models.VoigtModel(prefix='voigt_peak_7')
voigt_peak_8 = models.VoigtModel(prefix='voigt_peak_8')
voigt_peak_9 = models.VoigtModel(prefix='voigt_peak_9')
voigt_peak_10 = models.VoigtModel(prefix='voigt_peak_10')
voigt_peak_11 = models.VoigtModel(prefix='voigt_peak_11')
voigt_peak_12 = models.VoigtModel(prefix='voigt_peak_12')
voigt_peak_13 = models.VoigtModel(prefix='voigt_peak_13')
voigt_peak_14 = models.VoigtModel(prefix='voigt_peak_14')
voigt_peak_15 = models.VoigtModel(prefix='voigt_peak_15')
voigt_peak_16 = models.VoigtModel(prefix='voigt_peak_16')
voigt_peak_17 = models.VoigtModel(prefix='voigt_peak_17')
voigt_peak_18 = models.VoigtModel(prefix='voigt_peak_18')
voigt_peak_19 = models.VoigtModel(prefix='voigt_peak_19')
voigt_peak_20 = models.VoigtModel(prefix='voigt_peak_20')
voigt_peak_21 = models.VoigtModel(prefix='voigt_peak_21')
voigt_peak_22 = models.VoigtModel(prefix='voigt_peak_22')

In [129]:
peak_1_range_pulver = np.arange(np.where( 28.24 < data_pulver[0])[0][0],np.where( data_pulver[0] > 28.32)[0][0])
peak_2_range_pulver = np.arange(np.where( 28.335 < data_pulver[0])[0][0],np.where( data_pulver[0] > 28.4)[0][0])
peak_3_range_pulver = np.arange(np.where( 47.1 < data_pulver[0])[0][0],np.where( data_pulver[0] > 47.19)[0][0])
peak_4_range_pulver = np.arange(np.where( 47.24 < data_pulver[0])[0][0],np.where( data_pulver[0] > 47.31)[0][0])
peak_5_range_pulver = np.arange(np.where( 55.9 < data_pulver[0])[0][0],np.where( data_pulver[0] > 56.02)[0][0])
peak_6_range_pulver = np.arange(np.where( 56.06 < data_pulver[0])[0][0],np.where( data_pulver[0] > 56.17)[0][0])
peak_7_range_pulver = np.arange(np.where( 68.84 < data_pulver[0])[0][0],np.where( data_pulver[0] > 69.02)[0][0])
peak_8_range_pulver = np.arange(np.where( 69.12 < data_pulver[0])[0][0],np.where( data_pulver[0] > 69.22)[0][0])
peak_9_range_pulver = np.arange(np.where( 76.18 < data_pulver[0])[0][0],np.where( data_pulver[0] > 76.28)[0][0])
peak_10_range_pulver = np.arange(np.where( 76.38 < data_pulver[0])[0][0],np.where( data_pulver[0] > 76.52)[0][0])
peak_11_range_pulver = np.arange(np.where( 87.8 < data_pulver[0])[0][0],np.where( data_pulver[0] > 87.94)[0][0])
peak_12_range_pulver = np.arange(np.where( 88.08 < data_pulver[0])[0][0],np.where( data_pulver[0] > 88.22)[0][0])
peak_13_range_pulver = np.arange(np.where( 94.7 < data_pulver[0])[0][0],np.where( data_pulver[0] > 94.9)[0][0])
peak_14_range_pulver = np.arange(np.where( 95.04 < data_pulver[0])[0][0],np.where( data_pulver[0] > 95.18)[0][0])
peak_15_range_pulver = np.arange(np.where( 106.4 < data_pulver[0])[0][0],np.where( data_pulver[0] > 106.7)[0][0])
peak_16_range_pulver = np.arange(np.where( 106.86 < data_pulver[0])[0][0],np.where( data_pulver[0] > 107.04)[0][0])
peak_17_range_pulver = np.arange(np.where( 113.82 < data_pulver[0])[0][0],np.where( data_pulver[0] > 114.05)[0][0])
peak_18_range_pulver = np.arange(np.where( 114.3 < data_pulver[0])[0][0],np.where( data_pulver[0] > 114.48)[0][0])
peak_19_range_pulver = np.arange(np.where( 127.3 < data_pulver[0])[0][0],np.where( data_pulver[0] > 127.54)[0][0])
peak_20_range_pulver = np.arange(np.where( 127.8 < data_pulver[0])[0][0],np.where( data_pulver[0] > 128.2)[0][0])
peak_21_range_pulver = np.arange(np.where( 136.65 < data_pulver[0])[0][0],np.where( data_pulver[0] > 136.85)[0][0])
peak_22_range_pulver = np.arange(np.where( 137.35 < data_pulver[0])[0][0],np.where( data_pulver[0] > 137.65)[0][0])

In [130]:
params_peak_1 = voigt_peak_1.guess(data_pulver[1][peak_1_range_pulver],x=data_pulver[0][peak_1_range_pulver])
params_peak_2 = voigt_peak_2.guess(data_pulver[1][peak_2_range_pulver],x=data_pulver[0][peak_2_range_pulver])
params_peak_3 = voigt_peak_3.guess(data_pulver[1][peak_3_range_pulver],x=data_pulver[0][peak_3_range_pulver])
params_peak_4 = voigt_peak_4.guess(data_pulver[1][peak_4_range_pulver],x=data_pulver[0][peak_4_range_pulver])
params_peak_5 = voigt_peak_5.guess(data_pulver[1][peak_5_range_pulver],x=data_pulver[0][peak_5_range_pulver])
params_peak_6 = voigt_peak_6.guess(data_pulver[1][peak_6_range_pulver],x=data_pulver[0][peak_6_range_pulver])
params_peak_7 = voigt_peak_7.guess(data_pulver[1][peak_7_range_pulver],x=data_pulver[0][peak_7_range_pulver])
params_peak_8 = voigt_peak_8.guess(data_pulver[1][peak_8_range_pulver],x=data_pulver[0][peak_8_range_pulver])
params_peak_9 = voigt_peak_9.guess(data_pulver[1][peak_9_range_pulver],x=data_pulver[0][peak_9_range_pulver])
params_peak_10 = voigt_peak_10.guess(data_pulver[1][peak_10_range_pulver],x=data_pulver[0][peak_10_range_pulver])
params_peak_11 = voigt_peak_11.guess(data_pulver[1][peak_11_range_pulver],x=data_pulver[0][peak_11_range_pulver])
params_peak_12 = voigt_peak_12.guess(data_pulver[1][peak_12_range_pulver],x=data_pulver[0][peak_12_range_pulver])
params_peak_13 = voigt_peak_13.guess(data_pulver[1][peak_13_range_pulver],x=data_pulver[0][peak_13_range_pulver])
params_peak_14 = voigt_peak_14.guess(data_pulver[1][peak_14_range_pulver],x=data_pulver[0][peak_14_range_pulver])
params_peak_15 = voigt_peak_15.guess(data_pulver[1][peak_15_range_pulver],x=data_pulver[0][peak_15_range_pulver])
params_peak_16 = voigt_peak_16.guess(data_pulver[1][peak_16_range_pulver],x=data_pulver[0][peak_16_range_pulver])
params_peak_17 = voigt_peak_17.guess(data_pulver[1][peak_17_range_pulver],x=data_pulver[0][peak_17_range_pulver])
params_peak_18 = voigt_peak_18.guess(data_pulver[1][peak_18_range_pulver],x=data_pulver[0][peak_18_range_pulver])
params_peak_19 = voigt_peak_19.guess(data_pulver[1][peak_19_range_pulver],x=data_pulver[0][peak_19_range_pulver])
params_peak_20 = voigt_peak_20.guess(data_pulver[1][peak_20_range_pulver],x=data_pulver[0][peak_20_range_pulver])
params_peak_21 = voigt_peak_21.guess(data_pulver[1][peak_21_range_pulver],x=data_pulver[0][peak_21_range_pulver])
params_peak_22 = voigt_peak_22.guess(data_pulver[1][peak_22_range_pulver],x=data_pulver[0][peak_22_range_pulver])

In [131]:
params_peak_1['voigt_peak_1gamma'].set(value = params_peak_1.valuesdict()['voigt_peak_1sigma']/200 ,vary=True, expr='')
params_peak_2['voigt_peak_2gamma'].set(value = params_peak_2.valuesdict()['voigt_peak_2sigma']/200 ,vary=True, expr='')
params_peak_3['voigt_peak_3gamma'].set(value = params_peak_3.valuesdict()['voigt_peak_3sigma']/200 ,vary=True, expr='')
params_peak_4['voigt_peak_4gamma'].set(value = params_peak_4.valuesdict()['voigt_peak_4sigma']/200 ,vary=True, expr='')
params_peak_5['voigt_peak_5gamma'].set(value = params_peak_5.valuesdict()['voigt_peak_5sigma']/200 ,vary=True, expr='')
params_peak_6['voigt_peak_6gamma'].set(value = params_peak_6.valuesdict()['voigt_peak_6sigma']/200 ,vary=True, expr='')
params_peak_7['voigt_peak_7gamma'].set(value = params_peak_7.valuesdict()['voigt_peak_7sigma']/200 ,vary=True, expr='')
params_peak_8['voigt_peak_8gamma'].set(value = params_peak_8.valuesdict()['voigt_peak_8sigma']/200 ,vary=True, expr='')
params_peak_9['voigt_peak_9gamma'].set(value = params_peak_9.valuesdict()['voigt_peak_9sigma']/200 ,vary=True, expr='')
params_peak_10['voigt_peak_10gamma'].set(value = params_peak_10.valuesdict()['voigt_peak_10sigma']/200 ,vary=True, expr='')
params_peak_11['voigt_peak_11gamma'].set(value = params_peak_11.valuesdict()['voigt_peak_11sigma']/200 ,vary=True, expr='')
params_peak_12['voigt_peak_12gamma'].set(value = params_peak_12.valuesdict()['voigt_peak_12sigma']/200 ,vary=True, expr='')
params_peak_13['voigt_peak_13gamma'].set(value = params_peak_13.valuesdict()['voigt_peak_13sigma']/200 ,vary=True, expr='')
params_peak_14['voigt_peak_14gamma'].set(value = params_peak_14.valuesdict()['voigt_peak_14sigma']/200 ,vary=True, expr='')
params_peak_15['voigt_peak_15gamma'].set(value = params_peak_15.valuesdict()['voigt_peak_15sigma']/200 ,vary=True, expr='')
params_peak_16['voigt_peak_16gamma'].set(value = params_peak_16.valuesdict()['voigt_peak_16sigma']/200 ,vary=True, expr='')
params_peak_17['voigt_peak_17gamma'].set(value = params_peak_17.valuesdict()['voigt_peak_17sigma']/200 ,vary=True, expr='')
params_peak_18['voigt_peak_18gamma'].set(value = params_peak_18.valuesdict()['voigt_peak_18sigma']/200 ,vary=True, expr='')
params_peak_19['voigt_peak_19gamma'].set(value = params_peak_19.valuesdict()['voigt_peak_19sigma']/200 ,vary=True, expr='')
params_peak_20['voigt_peak_20gamma'].set(value = params_peak_20.valuesdict()['voigt_peak_20sigma']/200 ,vary=True, expr='')
params_peak_21['voigt_peak_21gamma'].set(value = params_peak_21.valuesdict()['voigt_peak_21sigma']/200 ,vary=True, expr='')
params_peak_22['voigt_peak_22gamma'].set(value = params_peak_22.valuesdict()['voigt_peak_22sigma']/200 ,vary=True, expr='')

In [132]:
#ab hier wiederholschleife
params_peak_1['voigt_peak_1sigma'].set(value = params_peak_1.valuesdict()['voigt_peak_1sigma'] ,vary=True, expr='')
params_peak_2['voigt_peak_2sigma'].set(value = params_peak_2.valuesdict()['voigt_peak_2sigma'] ,vary=True, expr='')
params_peak_3['voigt_peak_3sigma'].set(value = params_peak_3.valuesdict()['voigt_peak_3sigma'] ,vary=True, expr='')
params_peak_4['voigt_peak_4sigma'].set(value = params_peak_4.valuesdict()['voigt_peak_4sigma'] ,vary=True, expr='')
params_peak_5['voigt_peak_5sigma'].set(value = params_peak_5.valuesdict()['voigt_peak_5sigma'] ,vary=True, expr='')
params_peak_6['voigt_peak_6sigma'].set(value = params_peak_6.valuesdict()['voigt_peak_6sigma'] ,vary=True, expr='')
params_peak_7['voigt_peak_7sigma'].set(value = params_peak_7.valuesdict()['voigt_peak_7sigma'] ,vary=True, expr='')
params_peak_8['voigt_peak_8sigma'].set(value = params_peak_8.valuesdict()['voigt_peak_8sigma'] ,vary=True, expr='')
params_peak_9['voigt_peak_9sigma'].set(value = params_peak_9.valuesdict()['voigt_peak_9sigma'] ,vary=True, expr='')
params_peak_10['voigt_peak_10sigma'].set(value = params_peak_10.valuesdict()['voigt_peak_10sigma'] ,vary=True, expr='')
params_peak_11['voigt_peak_11sigma'].set(value = params_peak_11.valuesdict()['voigt_peak_11sigma'] ,vary=True, expr='')
params_peak_12['voigt_peak_12sigma'].set(value = params_peak_12.valuesdict()['voigt_peak_12sigma'] ,vary=True, expr='')
params_peak_13['voigt_peak_13sigma'].set(value = params_peak_13.valuesdict()['voigt_peak_13sigma'] ,vary=True, expr='')
params_peak_14['voigt_peak_14sigma'].set(value = params_peak_14.valuesdict()['voigt_peak_14sigma'] ,vary=True, expr='')
params_peak_15['voigt_peak_15sigma'].set(value = params_peak_15.valuesdict()['voigt_peak_15sigma'] ,vary=True, expr='')
params_peak_16['voigt_peak_16sigma'].set(value = params_peak_16.valuesdict()['voigt_peak_16sigma'] ,vary=True, expr='')
params_peak_17['voigt_peak_17sigma'].set(value = params_peak_17.valuesdict()['voigt_peak_17sigma'] ,vary=True, expr='')
params_peak_18['voigt_peak_18sigma'].set(value = params_peak_18.valuesdict()['voigt_peak_18sigma'] ,vary=True, expr='')
params_peak_19['voigt_peak_19sigma'].set(value = params_peak_19.valuesdict()['voigt_peak_19sigma'] ,vary=True, expr='')
params_peak_20['voigt_peak_20sigma'].set(value = params_peak_20.valuesdict()['voigt_peak_20sigma'] ,vary=True, expr='')
params_peak_21['voigt_peak_21sigma'].set(value = params_peak_21.valuesdict()['voigt_peak_21sigma'] ,vary=True, expr='')
params_peak_22['voigt_peak_22sigma'].set(value = params_peak_22.valuesdict()['voigt_peak_22sigma'] ,vary=True, expr='')

params_peak_1['voigt_peak_1gamma'].set(value = params_peak_1.valuesdict()['voigt_peak_1gamma'] ,vary=True, expr='')
params_peak_2['voigt_peak_2gamma'].set(value = params_peak_2.valuesdict()['voigt_peak_2gamma'] ,vary=True, expr='')
params_peak_3['voigt_peak_3gamma'].set(value = params_peak_3.valuesdict()['voigt_peak_3gamma'] ,vary=True, expr='')
params_peak_4['voigt_peak_4gamma'].set(value = params_peak_4.valuesdict()['voigt_peak_4gamma'] ,vary=True, expr='')
params_peak_5['voigt_peak_5gamma'].set(value = params_peak_5.valuesdict()['voigt_peak_5gamma'] ,vary=True, expr='')
params_peak_6['voigt_peak_6gamma'].set(value = params_peak_6.valuesdict()['voigt_peak_6gamma'] ,vary=True, expr='')
params_peak_7['voigt_peak_7gamma'].set(value = params_peak_7.valuesdict()['voigt_peak_7gamma'] ,vary=True, expr='')
params_peak_8['voigt_peak_8gamma'].set(value = params_peak_8.valuesdict()['voigt_peak_8gamma'] ,vary=True, expr='')
params_peak_9['voigt_peak_9gamma'].set(value = params_peak_9.valuesdict()['voigt_peak_9gamma'] ,vary=True, expr='')
params_peak_10['voigt_peak_10gamma'].set(value = params_peak_10.valuesdict()['voigt_peak_10gamma'] ,vary=True, expr='')
params_peak_11['voigt_peak_11gamma'].set(value = params_peak_11.valuesdict()['voigt_peak_11gamma'] ,vary=True, expr='')
params_peak_12['voigt_peak_12gamma'].set(value = params_peak_12.valuesdict()['voigt_peak_12gamma'] ,vary=True, expr='')
params_peak_13['voigt_peak_13gamma'].set(value = params_peak_13.valuesdict()['voigt_peak_13gamma'] ,vary=True, expr='')
params_peak_14['voigt_peak_14gamma'].set(value = params_peak_14.valuesdict()['voigt_peak_14gamma'] ,vary=True, expr='')
params_peak_15['voigt_peak_15gamma'].set(value = params_peak_15.valuesdict()['voigt_peak_15gamma'] ,vary=True, expr='')
params_peak_16['voigt_peak_16gamma'].set(value = params_peak_16.valuesdict()['voigt_peak_16gamma'] ,vary=True, expr='')
params_peak_17['voigt_peak_17gamma'].set(value = params_peak_17.valuesdict()['voigt_peak_17gamma'] ,vary=True, expr='')
params_peak_18['voigt_peak_18gamma'].set(value = params_peak_18.valuesdict()['voigt_peak_18gamma'] ,vary=True, expr='')
params_peak_19['voigt_peak_19gamma'].set(value = params_peak_19.valuesdict()['voigt_peak_19gamma'] ,vary=True, expr='')
params_peak_20['voigt_peak_20gamma'].set(value = params_peak_20.valuesdict()['voigt_peak_20gamma'] ,vary=True, expr='')
params_peak_21['voigt_peak_21gamma'].set(value = params_peak_21.valuesdict()['voigt_peak_21gamma'] ,vary=True, expr='')
params_peak_22['voigt_peak_22gamma'].set(value = params_peak_22.valuesdict()['voigt_peak_22gamma'] ,vary=True, expr='')

In [133]:
multi_voigt_pulver = voigt_peak_1 + voigt_peak_2 + voigt_peak_3 + voigt_peak_4 + voigt_peak_5 + voigt_peak_6 + voigt_peak_7 + voigt_peak_8 + voigt_peak_9 + voigt_peak_10 + voigt_peak_11 + voigt_peak_12 + voigt_peak_13 + voigt_peak_14 + voigt_peak_15 + voigt_peak_16 + voigt_peak_17 + voigt_peak_18 + voigt_peak_19 + voigt_peak_20 + voigt_peak_21 + voigt_peak_22
params_multivoigt_pulver = multi_voigt_pulver.make_params()
params_multivoigt_pulver.update(params_peak_1)
params_multivoigt_pulver.update(params_peak_2)
params_multivoigt_pulver.update(params_peak_3)
params_multivoigt_pulver.update(params_peak_4)
params_multivoigt_pulver.update(params_peak_5)
params_multivoigt_pulver.update(params_peak_6)
params_multivoigt_pulver.update(params_peak_7)
params_multivoigt_pulver.update(params_peak_8)
params_multivoigt_pulver.update(params_peak_9)
params_multivoigt_pulver.update(params_peak_10)
params_multivoigt_pulver.update(params_peak_11)
params_multivoigt_pulver.update(params_peak_12)
params_multivoigt_pulver.update(params_peak_13)
params_multivoigt_pulver.update(params_peak_14)
params_multivoigt_pulver.update(params_peak_15)
params_multivoigt_pulver.update(params_peak_16)
params_multivoigt_pulver.update(params_peak_17)
params_multivoigt_pulver.update(params_peak_18)
params_multivoigt_pulver.update(params_peak_19)
params_multivoigt_pulver.update(params_peak_20)
params_multivoigt_pulver.update(params_peak_21)
params_multivoigt_pulver.update(params_peak_22)

In [134]:
#nerv1 = 1/np.sqrt(data_pulver[1])
#for i in range(0,len(nerv1),1):
#    if nerv1[i] > 0.5:
#        nerv1[i] = 0.5
fit_multi_voigt_pulver = multi_voigt_pulver.fit(data_pulver[1][np.where(data_pulver[1] > 0)],params_multivoigt_pulver,x=data_pulver[0][np.where(data_pulver[1] > 0)], weights=1/np.sqrt(data_pulver[1][np.where(data_pulver[1] > 0)]))
print(fit_multi_voigt_pulver.fit_report())

KeyboardInterrupt: 

In [ ]:
fit_multi_voigt_pulver.plot()
plt.show()

In [121]:
plt.errorbar(data_pulver[0],data_pulver[1],np.sqrt(data_pulver[1]),label = r'Messdaten mit Fehler $\sqrt{N}$')
plt.plot(np.arange(data_pulver[0][0], data_pulver[0][-1],0.001),fit_multi_voigt_pulver.eval(x=np.arange(data_pulver[0][0], data_pulver[0][-1],0.001)),label = r'Voigtfit')
plt.xticks(size = 24)
plt.yticks(size = 24)
plt.xlim(10,143)
plt.legend(loc = 'best', fontsize = 24)
plt.ylabel('Counts', size = 24)
plt.xlabel(r'2$\theta/^{\circ}$', size = 24)
plt.show()

In [74]:
Silicium_Simulation = np.loadtxt("Silicium.x_y").T

In [75]:
Silicium_Simulation

array([[  2.00000000e+01,   2.00100000e+01,   2.00200000e+01, ...,
          1.39976000e+02,   1.39986000e+02,   1.39996000e+02],
       [  1.34000000e-01,   1.34000000e-01,   1.35000000e-01, ...,
          1.56000000e-01,   1.55000000e-01,   1.54000000e-01]])

In [76]:
voigt_peak_1ss = models.VoigtModel(prefix='voigt_peak_1')
voigt_peak_2ss = models.VoigtModel(prefix='voigt_peak_2')
voigt_peak_3ss = models.VoigtModel(prefix='voigt_peak_3')
voigt_peak_4ss = models.VoigtModel(prefix='voigt_peak_4')
voigt_peak_5ss = models.VoigtModel(prefix='voigt_peak_5')
voigt_peak_6ss = models.VoigtModel(prefix='voigt_peak_6')
voigt_peak_7ss = models.VoigtModel(prefix='voigt_peak_7')
voigt_peak_8ss = models.VoigtModel(prefix='voigt_peak_8')
voigt_peak_9ss = models.VoigtModel(prefix='voigt_peak_9')
voigt_peak_10ss = models.VoigtModel(prefix='voigt_peak_10')
voigt_peak_11ss = models.VoigtModel(prefix='voigt_peak_11')
voigt_peak_12ss = models.VoigtModel(prefix='voigt_peak_12')
voigt_peak_13ss = models.VoigtModel(prefix='voigt_peak_13')
voigt_peak_14ss = models.VoigtModel(prefix='voigt_peak_14')
voigt_peak_15ss = models.VoigtModel(prefix='voigt_peak_15')
voigt_peak_16ss = models.VoigtModel(prefix='voigt_peak_16')
voigt_peak_17ss = models.VoigtModel(prefix='voigt_peak_17')
voigt_peak_18ss = models.VoigtModel(prefix='voigt_peak_18')
voigt_peak_19ss = models.VoigtModel(prefix='voigt_peak_19')
voigt_peak_20ss = models.VoigtModel(prefix='voigt_peak_20')
voigt_peak_21ss = models.VoigtModel(prefix='voigt_peak_21')
voigt_peak_22ss = models.VoigtModel(prefix='voigt_peak_22')

In [77]:
peak_1_range_pulverss = np.arange(np.where( 28.24+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 28.32+0.15)[0][0])
peak_2_range_pulverss = np.arange(np.where( 28.335+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 28.4+0.15)[0][0])
peak_3_range_pulverss = np.arange(np.where( 47.1+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 47.19+0.15)[0][0])
peak_4_range_pulverss = np.arange(np.where( 47.24+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 47.31+0.15)[0][0])
peak_5_range_pulverss = np.arange(np.where( 55.9+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 56.02+0.15)[0][0])
peak_6_range_pulverss = np.arange(np.where( 56.06+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 56.17+0.15)[0][0])
peak_7_range_pulverss = np.arange(np.where( 68.84+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 69.02+0.15)[0][0])
peak_8_range_pulverss = np.arange(np.where( 69.12+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 69.22+0.15)[0][0])
peak_9_range_pulverss = np.arange(np.where( 76.18+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 76.28+0.15)[0][0])
peak_10_range_pulverss = np.arange(np.where( 76.38+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 76.52+0.15)[0][0])
peak_11_range_pulverss = np.arange(np.where( 87.8+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 87.94+0.15)[0][0])
peak_12_range_pulverss = np.arange(np.where( 88.08+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 88.22+0.15)[0][0])
peak_13_range_pulverss = np.arange(np.where( 94.7+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 94.9+0.15)[0][0])
peak_14_range_pulverss = np.arange(np.where( 95.04+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 95.18+0.15)[0][0])
peak_15_range_pulverss = np.arange(np.where( 106.4+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 106.7+0.15)[0][0])
peak_16_range_pulverss = np.arange(np.where( 106.86+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 107.04+0.15)[0][0])
peak_17_range_pulverss = np.arange(np.where( 113.82+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 114.05+0.15)[0][0])
peak_18_range_pulverss = np.arange(np.where( 114.3+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 114.48+0.15)[0][0])
peak_19_range_pulverss = np.arange(np.where( 127.3+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 127.54+0.15)[0][0])
peak_20_range_pulverss = np.arange(np.where( 127.9+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 128.08+0.15)[0][0])
peak_21_range_pulverss = np.arange(np.where( 136.65+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 136.85+0.15)[0][0])
peak_22_range_pulverss = np.arange(np.where( 137.35+0.15 < Silicium_Simulation[0])[0][0],np.where( Silicium_Simulation[0] > 137.65+0.15)[0][0])

In [78]:
params_peak_1ss = voigt_peak_1ss.guess(Silicium_Simulation[1][peak_1_range_pulverss],x=Silicium_Simulation[0][peak_1_range_pulverss])
params_peak_2ss = voigt_peak_2ss.guess(Silicium_Simulation[1][peak_2_range_pulverss],x=Silicium_Simulation[0][peak_2_range_pulverss])
params_peak_3ss = voigt_peak_3ss.guess(Silicium_Simulation[1][peak_3_range_pulverss],x=Silicium_Simulation[0][peak_3_range_pulverss])
params_peak_4ss = voigt_peak_4ss.guess(Silicium_Simulation[1][peak_4_range_pulverss],x=Silicium_Simulation[0][peak_4_range_pulverss])
params_peak_5ss = voigt_peak_5ss.guess(Silicium_Simulation[1][peak_5_range_pulverss],x=Silicium_Simulation[0][peak_5_range_pulverss])
params_peak_6ss = voigt_peak_6ss.guess(Silicium_Simulation[1][peak_6_range_pulverss],x=Silicium_Simulation[0][peak_6_range_pulverss])
params_peak_7ss = voigt_peak_7ss.guess(Silicium_Simulation[1][peak_7_range_pulverss],x=Silicium_Simulation[0][peak_7_range_pulverss])
params_peak_8ss = voigt_peak_8ss.guess(Silicium_Simulation[1][peak_8_range_pulverss],x=Silicium_Simulation[0][peak_8_range_pulverss])
params_peak_9ss = voigt_peak_9ss.guess(Silicium_Simulation[1][peak_9_range_pulverss],x=Silicium_Simulation[0][peak_9_range_pulverss])
params_peak_10ss = voigt_peak_10ss.guess(Silicium_Simulation[1][peak_10_range_pulverss],x=Silicium_Simulation[0][peak_10_range_pulverss])
params_peak_11ss = voigt_peak_11ss.guess(Silicium_Simulation[1][peak_11_range_pulverss],x=Silicium_Simulation[0][peak_11_range_pulverss])
params_peak_12ss = voigt_peak_12ss.guess(Silicium_Simulation[1][peak_12_range_pulverss],x=Silicium_Simulation[0][peak_12_range_pulverss])
params_peak_13ss = voigt_peak_13ss.guess(Silicium_Simulation[1][peak_13_range_pulverss],x=Silicium_Simulation[0][peak_13_range_pulverss])
params_peak_14ss = voigt_peak_14ss.guess(Silicium_Simulation[1][peak_14_range_pulverss],x=Silicium_Simulation[0][peak_14_range_pulverss])
params_peak_15ss = voigt_peak_15ss.guess(Silicium_Simulation[1][peak_15_range_pulverss],x=Silicium_Simulation[0][peak_15_range_pulverss])
params_peak_16ss = voigt_peak_16ss.guess(Silicium_Simulation[1][peak_16_range_pulverss],x=Silicium_Simulation[0][peak_16_range_pulverss])
params_peak_17ss = voigt_peak_17ss.guess(Silicium_Simulation[1][peak_17_range_pulverss],x=Silicium_Simulation[0][peak_17_range_pulverss])
params_peak_18ss = voigt_peak_18ss.guess(Silicium_Simulation[1][peak_18_range_pulverss],x=Silicium_Simulation[0][peak_18_range_pulverss])
params_peak_19ss = voigt_peak_19ss.guess(Silicium_Simulation[1][peak_19_range_pulverss],x=Silicium_Simulation[0][peak_19_range_pulverss])
params_peak_20ss = voigt_peak_20ss.guess(Silicium_Simulation[1][peak_20_range_pulverss],x=Silicium_Simulation[0][peak_20_range_pulverss])
params_peak_21ss = voigt_peak_21ss.guess(Silicium_Simulation[1][peak_21_range_pulverss],x=Silicium_Simulation[0][peak_21_range_pulverss])
params_peak_22ss = voigt_peak_22ss.guess(Silicium_Simulation[1][peak_22_range_pulverss],x=Silicium_Simulation[0][peak_22_range_pulverss])

In [79]:
params_peak_1ss['voigt_peak_1gamma'].set(value = params_peak_1ss.valuesdict()['voigt_peak_1sigma']/200 ,vary=True, expr='')
params_peak_2ss['voigt_peak_2gamma'].set(value = params_peak_2ss.valuesdict()['voigt_peak_2sigma']/200 ,vary=True, expr='')
params_peak_3ss['voigt_peak_3gamma'].set(value = params_peak_3ss.valuesdict()['voigt_peak_3sigma']/200 ,vary=True, expr='')
params_peak_4ss['voigt_peak_4gamma'].set(value = params_peak_4ss.valuesdict()['voigt_peak_4sigma']/200 ,vary=True, expr='')
params_peak_5ss['voigt_peak_5gamma'].set(value = params_peak_5ss.valuesdict()['voigt_peak_5sigma']/200 ,vary=True, expr='')
params_peak_6ss['voigt_peak_6gamma'].set(value = params_peak_6ss.valuesdict()['voigt_peak_6sigma']/200 ,vary=True, expr='')
params_peak_7ss['voigt_peak_7gamma'].set(value = params_peak_7ss.valuesdict()['voigt_peak_7sigma']/200 ,vary=True, expr='')
params_peak_8ss['voigt_peak_8gamma'].set(value = params_peak_8ss.valuesdict()['voigt_peak_8sigma']/200 ,vary=True, expr='')
params_peak_9ss['voigt_peak_9gamma'].set(value = params_peak_9ss.valuesdict()['voigt_peak_9sigma']/200 ,vary=True, expr='')
params_peak_10ss['voigt_peak_10gamma'].set(value = params_peak_10ss.valuesdict()['voigt_peak_10sigma']/200 ,vary=True, expr='')
params_peak_11ss['voigt_peak_11gamma'].set(value = params_peak_11ss.valuesdict()['voigt_peak_11sigma']/200 ,vary=True, expr='')
params_peak_12ss['voigt_peak_12gamma'].set(value = params_peak_12ss.valuesdict()['voigt_peak_12sigma']/200 ,vary=True, expr='')
params_peak_13ss['voigt_peak_13gamma'].set(value = params_peak_13ss.valuesdict()['voigt_peak_13sigma']/200 ,vary=True, expr='')
params_peak_14ss['voigt_peak_14gamma'].set(value = params_peak_14ss.valuesdict()['voigt_peak_14sigma']/200 ,vary=True, expr='')
params_peak_15ss['voigt_peak_15gamma'].set(value = params_peak_15ss.valuesdict()['voigt_peak_15sigma']/200 ,vary=True, expr='')
params_peak_16ss['voigt_peak_16gamma'].set(value = params_peak_16ss.valuesdict()['voigt_peak_16sigma']/200 ,vary=True, expr='')
params_peak_17ss['voigt_peak_17gamma'].set(value = params_peak_17ss.valuesdict()['voigt_peak_17sigma']/200 ,vary=True, expr='')
params_peak_18ss['voigt_peak_18gamma'].set(value = params_peak_18ss.valuesdict()['voigt_peak_18sigma']/200 ,vary=True, expr='')
params_peak_19ss['voigt_peak_19gamma'].set(value = params_peak_19ss.valuesdict()['voigt_peak_19sigma']/200 ,vary=True, expr='')
params_peak_20ss['voigt_peak_20gamma'].set(value = params_peak_20ss.valuesdict()['voigt_peak_20sigma']/200 ,vary=True, expr='')
params_peak_21ss['voigt_peak_21gamma'].set(value = params_peak_21ss.valuesdict()['voigt_peak_21sigma']/200 ,vary=True, expr='')
params_peak_22ss['voigt_peak_22gamma'].set(value = params_peak_22ss.valuesdict()['voigt_peak_22sigma']/200 ,vary=True, expr='')

In [80]:
#ab hier wiederholschleife
params_peak_1ss['voigt_peak_1sigma'].set(value = params_peak_1ss.valuesdict()['voigt_peak_1sigma'] ,vary=True, expr='')
params_peak_2ss['voigt_peak_2sigma'].set(value = params_peak_2ss.valuesdict()['voigt_peak_2sigma'] ,vary=True, expr='')
params_peak_3ss['voigt_peak_3sigma'].set(value = params_peak_3ss.valuesdict()['voigt_peak_3sigma'] ,vary=True, expr='')
params_peak_4ss['voigt_peak_4sigma'].set(value = params_peak_4ss.valuesdict()['voigt_peak_4sigma'] ,vary=True, expr='')
params_peak_5ss['voigt_peak_5sigma'].set(value = params_peak_5ss.valuesdict()['voigt_peak_5sigma'] ,vary=True, expr='')
params_peak_6ss['voigt_peak_6sigma'].set(value = params_peak_6ss.valuesdict()['voigt_peak_6sigma'] ,vary=True, expr='')
params_peak_7ss['voigt_peak_7sigma'].set(value = params_peak_7ss.valuesdict()['voigt_peak_7sigma'] ,vary=True, expr='')
params_peak_8ss['voigt_peak_8sigma'].set(value = params_peak_8ss.valuesdict()['voigt_peak_8sigma'] ,vary=True, expr='')
params_peak_9ss['voigt_peak_9sigma'].set(value = params_peak_9ss.valuesdict()['voigt_peak_9sigma'] ,vary=True, expr='')
params_peak_10ss['voigt_peak_10sigma'].set(value = params_peak_10ss.valuesdict()['voigt_peak_10sigma'] ,vary=True, expr='')
params_peak_11ss['voigt_peak_11sigma'].set(value = params_peak_11ss.valuesdict()['voigt_peak_11sigma'] ,vary=True, expr='')
params_peak_12ss['voigt_peak_12sigma'].set(value = params_peak_12ss.valuesdict()['voigt_peak_12sigma'] ,vary=True, expr='')
params_peak_13ss['voigt_peak_13sigma'].set(value = params_peak_13ss.valuesdict()['voigt_peak_13sigma'] ,vary=True, expr='')
params_peak_14ss['voigt_peak_14sigma'].set(value = params_peak_14ss.valuesdict()['voigt_peak_14sigma'] ,vary=True, expr='')
params_peak_15ss['voigt_peak_15sigma'].set(value = params_peak_15ss.valuesdict()['voigt_peak_15sigma'] ,vary=True, expr='')
params_peak_16ss['voigt_peak_16sigma'].set(value = params_peak_16ss.valuesdict()['voigt_peak_16sigma'] ,vary=True, expr='')
params_peak_17ss['voigt_peak_17sigma'].set(value = params_peak_17ss.valuesdict()['voigt_peak_17sigma'] ,vary=True, expr='')
params_peak_18ss['voigt_peak_18sigma'].set(value = params_peak_18ss.valuesdict()['voigt_peak_18sigma'] ,vary=True, expr='')
params_peak_19ss['voigt_peak_19sigma'].set(value = params_peak_19ss.valuesdict()['voigt_peak_19sigma'] ,vary=True, expr='')
params_peak_20ss['voigt_peak_20sigma'].set(value = params_peak_20ss.valuesdict()['voigt_peak_20sigma'] ,vary=True, expr='')
params_peak_21ss['voigt_peak_21sigma'].set(value = params_peak_21ss.valuesdict()['voigt_peak_21sigma'] ,vary=True, expr='')
params_peak_22ss['voigt_peak_22sigma'].set(value = params_peak_22ss.valuesdict()['voigt_peak_22sigma'] ,vary=True, expr='')

params_peak_1ss['voigt_peak_1gamma'].set(value = params_peak_1ss.valuesdict()['voigt_peak_1gamma'] ,vary=True, expr='')
params_peak_2ss['voigt_peak_2gamma'].set(value = params_peak_2ss.valuesdict()['voigt_peak_2gamma'] ,vary=True, expr='')
params_peak_3ss['voigt_peak_3gamma'].set(value = params_peak_3ss.valuesdict()['voigt_peak_3gamma'] ,vary=True, expr='')
params_peak_4ss['voigt_peak_4gamma'].set(value = params_peak_4ss.valuesdict()['voigt_peak_4gamma'] ,vary=True, expr='')
params_peak_5ss['voigt_peak_5gamma'].set(value = params_peak_5ss.valuesdict()['voigt_peak_5gamma'] ,vary=True, expr='')
params_peak_6ss['voigt_peak_6gamma'].set(value = params_peak_6ss.valuesdict()['voigt_peak_6gamma'] ,vary=True, expr='')
params_peak_7ss['voigt_peak_7gamma'].set(value = params_peak_7ss.valuesdict()['voigt_peak_7gamma'] ,vary=True, expr='')
params_peak_8ss['voigt_peak_8gamma'].set(value = params_peak_8ss.valuesdict()['voigt_peak_8gamma'] ,vary=True, expr='')
params_peak_9ss['voigt_peak_9gamma'].set(value = params_peak_9ss.valuesdict()['voigt_peak_9gamma'] ,vary=True, expr='')
params_peak_10ss['voigt_peak_10gamma'].set(value = params_peak_10ss.valuesdict()['voigt_peak_10gamma'] ,vary=True, expr='')
params_peak_11ss['voigt_peak_11gamma'].set(value = params_peak_11ss.valuesdict()['voigt_peak_11gamma'] ,vary=True, expr='')
params_peak_12ss['voigt_peak_12gamma'].set(value = params_peak_12ss.valuesdict()['voigt_peak_12gamma'] ,vary=True, expr='')
params_peak_13ss['voigt_peak_13gamma'].set(value = params_peak_13ss.valuesdict()['voigt_peak_13gamma'] ,vary=True, expr='')
params_peak_14ss['voigt_peak_14gamma'].set(value = params_peak_14ss.valuesdict()['voigt_peak_14gamma'] ,vary=True, expr='')
params_peak_15ss['voigt_peak_15gamma'].set(value = params_peak_15ss.valuesdict()['voigt_peak_15gamma'] ,vary=True, expr='')
params_peak_16ss['voigt_peak_16gamma'].set(value = params_peak_16ss.valuesdict()['voigt_peak_16gamma'] ,vary=True, expr='')
params_peak_17ss['voigt_peak_17gamma'].set(value = params_peak_17ss.valuesdict()['voigt_peak_17gamma'] ,vary=True, expr='')
params_peak_18ss['voigt_peak_18gamma'].set(value = params_peak_18ss.valuesdict()['voigt_peak_18gamma'] ,vary=True, expr='')
params_peak_19ss['voigt_peak_19gamma'].set(value = params_peak_19ss.valuesdict()['voigt_peak_19gamma'] ,vary=True, expr='')
params_peak_20ss['voigt_peak_20gamma'].set(value = params_peak_20ss.valuesdict()['voigt_peak_20gamma'] ,vary=True, expr='')
params_peak_21ss['voigt_peak_21gamma'].set(value = params_peak_21ss.valuesdict()['voigt_peak_21gamma'] ,vary=True, expr='')
params_peak_22ss['voigt_peak_22gamma'].set(value = params_peak_22ss.valuesdict()['voigt_peak_22gamma'] ,vary=True, expr='')

In [81]:
multi_voigt_pulverss = voigt_peak_1ss + voigt_peak_2ss + voigt_peak_3ss + voigt_peak_4ss + voigt_peak_5ss + voigt_peak_6ss + voigt_peak_7ss + voigt_peak_8ss + voigt_peak_9ss + voigt_peak_10ss + voigt_peak_11ss + voigt_peak_12ss + voigt_peak_13ss + voigt_peak_14ss + voigt_peak_15ss + voigt_peak_16ss + voigt_peak_17ss + voigt_peak_18ss + voigt_peak_19ss + voigt_peak_20ss + voigt_peak_21ss + voigt_peak_22ss 
params_multivoigt_pulverss = multi_voigt_pulverss.make_params()
params_multivoigt_pulverss.update(params_peak_1ss)
params_multivoigt_pulverss.update(params_peak_2ss)
params_multivoigt_pulverss.update(params_peak_3ss)
params_multivoigt_pulverss.update(params_peak_4ss)
params_multivoigt_pulverss.update(params_peak_5ss)
params_multivoigt_pulverss.update(params_peak_6ss)
params_multivoigt_pulverss.update(params_peak_7ss)
params_multivoigt_pulverss.update(params_peak_8ss)
params_multivoigt_pulverss.update(params_peak_9ss)
params_multivoigt_pulverss.update(params_peak_10ss)
params_multivoigt_pulverss.update(params_peak_11ss)
params_multivoigt_pulverss.update(params_peak_12ss)
params_multivoigt_pulverss.update(params_peak_13ss)
params_multivoigt_pulverss.update(params_peak_14ss)
params_multivoigt_pulverss.update(params_peak_15ss)
params_multivoigt_pulverss.update(params_peak_16ss)
params_multivoigt_pulverss.update(params_peak_17ss)
params_multivoigt_pulverss.update(params_peak_18ss)
params_multivoigt_pulverss.update(params_peak_19ss)
params_multivoigt_pulverss.update(params_peak_20ss)
params_multivoigt_pulverss.update(params_peak_21ss)
params_multivoigt_pulverss.update(params_peak_22ss)

In [82]:
fit_multi_voigt_pulverss = multi_voigt_pulverss.fit(Silicium_Simulation[1][np.where(Silicium_Simulation[1] > -1)],params_multivoigt_pulverss,x=Silicium_Simulation[0][np.where(Silicium_Simulation[1] > -1)])
print(fit_multi_voigt_pulverss.fit_report())

[[Model]]
    (((((((((((((((((((((Model(voigt, prefix='voigt_peak_1') + Model(voigt, prefix='voigt_peak_2')) + Model(voigt, prefix='voigt_peak_3')) + Model(voigt, prefix='voigt_peak_4')) + Model(voigt, prefix='voigt_peak_5')) + Model(voigt, prefix='voigt_peak_6')) + Model(voigt, prefix='voigt_peak_7')) + Model(voigt, prefix='voigt_peak_8')) + Model(voigt, prefix='voigt_peak_9')) + Model(voigt, prefix='voigt_peak_10')) + Model(voigt, prefix='voigt_peak_11')) + Model(voigt, prefix='voigt_peak_12')) + Model(voigt, prefix='voigt_peak_13')) + Model(voigt, prefix='voigt_peak_14')) + Model(voigt, prefix='voigt_peak_15')) + Model(voigt, prefix='voigt_peak_16')) + Model(voigt, prefix='voigt_peak_17')) + Model(voigt, prefix='voigt_peak_18')) + Model(voigt, prefix='voigt_peak_19')) + Model(voigt, prefix='voigt_peak_20')) + Model(voigt, prefix='voigt_peak_21')) + Model(voigt, prefix='voigt_peak_22'))
[[Fit Statistics]]
    # function evals   = 1786
    # data points      = 11979
    # variables  

In [83]:
sinep = []
sinep1 = []
sinep2 = []
for i in range(1,12,1):
    sinep2.append(params_multivoigt_pulverss.valuesdict()['voigt_peak_'+str(2*i-1)+'center'] - params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i-1)+'center'])
    sinep.append(params_multivoigt_pulverss.valuesdict()['voigt_peak_'+str(2*i)+'center'] - params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i)+'center'])
for i in range(1,23,1):
    sinep1.append(params_multivoigt_pulverss.valuesdict()['voigt_peak_'+str(i)+'center'] - params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(i)+'center'])
plt.bar(np.arange(1,12,1),sinep2, width = 1)
plt.plot(np.arange(1,13,1),np.arange(1,13,1)/np.arange(1,13,1)*(np.sum(sinep2)/len(sinep2)), '-r', label = 'Mittelwert bei '+str(round(np.sum(sinep2)/len(sinep2), 4)))
plt.legend(loc = 'best', fontsize = 24)
plt.xlim([1,12])
plt.ylim(0,np.max(sinep2))
plt.yticks(np.arange(0,np.max(sinep2)+0.02,0.02))
plt.ylabel(r'$\Delta 2\theta/^{\circ}$', size = 24)
plt.xlabel(r'Peak Nummer K$_{\alpha_1}$', size = 24)
plt.show()

plt.bar(np.arange(1,12,1),sinep, width = 1)
plt.plot(np.arange(1,13,1),np.arange(1,13,1)/np.arange(1,13,1)*(np.sum(sinep)/len(sinep)), '-r', label = 'Mittelwert bei '+str(round(np.sum(sinep)/len(sinep), 4)))
plt.legend(loc = 'best', fontsize = 24)
plt.yticks(np.arange(0,np.max(sinep)+0.02,0.02))
plt.xlim([1,12])
plt.ylim(0,np.max(sinep))
plt.ylabel(r'$\Delta 2\theta/^{\circ}$', size = 24)
plt.xlabel(r'Peak Nummer K$_{\alpha_2}$', size = 24)
plt.show()

plt.bar(np.arange(1,23,1),sinep1, width = 1)
plt.plot(np.arange(1,24,1),np.arange(1,24,1)/np.arange(1,24,1)*(np.sum(sinep1)/len(sinep1)), '-r', label = 'Mittelwert bei '+str(round(np.sum(sinep1)/len(sinep1), 4)))
plt.legend(loc = 'best', fontsize = 34)
plt.yticks(np.arange(0,np.max(sinep)+0.02,0.02))
plt.xlim([1,23])
plt.ylim(0,np.max(sinep))
plt.ylabel(r'$\Delta 2\theta/^{\circ}$', size = 34)
plt.xlabel(r'Peak Nummer nach Energie', size = 34)
plt.show()
print(np.sum(sinep1)/len(sinep1),np.sum(sinep)/len(sinep),np.sum(sinep2)/len(sinep2))

0.171169046027 0.169920125651 0.172417966403


In [84]:
mess_a1 = []
mess_a2 = []
theo_a1 = []
theo_a2 = []
for i in range(1,12,1):
    mess_a1.append(params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i-1)+'center'])
    mess_a2.append(params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i)+'center'])
    theo_a1.append(params_multivoigt_pulverss.valuesdict()['voigt_peak_'+str(2*i-1)+'center'])
    theo_a2.append(params_multivoigt_pulverss.valuesdict()['voigt_peak_'+str(2*i)+'center'])
E_a1m_a2m_a1t_a2t = [8052,8032.1,8047.8,8027.8]
mess_a1 = np.array(mess_a1)
mess_a2 = np.array(mess_a2)
theo_a1 = np.array(theo_a1)
theo_a2 = np.array(theo_a2)
nhkl_mess_a1 = (2*(np.sin(mess_a1/2*np.pi/180))*(E_a1m_a2m_a1t_a2t[0]*1.6021766208*10**(-19))*(543.1020504*10**(-12))/(6.62607004*10**(-34)*299792458))**2
nhkl_mess_a2 = (2*(np.sin(mess_a2/2*np.pi/180))*(E_a1m_a2m_a1t_a2t[1]*1.6021766208*10**(-19))*(543.1020504*10**(-12))/(6.62607004*10**(-34)*299792458))**2
nhkl_theo_a1 = (2*(np.sin(theo_a1/2*np.pi/180))*(E_a1m_a2m_a1t_a2t[2]*1.6021766208*10**(-19))*(543.1020504*10**(-12))/(6.62607004*10**(-34)*299792458))**2
nhkl_theo_a2 = (2*(np.sin(theo_a2/2*np.pi/180))*(E_a1m_a2m_a1t_a2t[3]*1.6021766208*10**(-19))*(543.1020504*10**(-12))/(6.62607004*10**(-34)*299792458))**2
print(nhkl_mess_a1,nhkl_mess_a2)
käse = [3,8,11,16,19,24,27,32,35,40,43]
print(543.1020504*10**(-12)/np.sqrt(käse),543.1020504*10**(-12)/np.sqrt(nhkl_mess_a1),543.1020504*10**(-12)/np.sqrt(nhkl_mess_a2))

[  2.97102377   7.95829415  10.95338234  15.95223836  18.9538067
  23.95729654  26.96536845  31.97464888  34.9781605   39.99657149
  43.00693418] [  2.97091649   7.95742248  10.95470842  15.95189105  18.95759298
  23.96261261  26.96533729  31.97404194  34.98467754  39.99595374
  43.00651595]
[  3.13560115e-10   1.92015571e-10   1.63751429e-10   1.35775513e-10
   1.24596155e-10   1.10860242e-10   1.04520038e-10   9.60077857e-11
   9.18010017e-11   8.58719741e-11   8.28222862e-11] [  3.15085472e-10   1.92518048e-10   1.64099523e-10   1.35978619e-10
   1.24747893e-10   1.10959001e-10   1.04587134e-10   9.60458381e-11
   9.18296564e-11   8.58756545e-11   8.28156091e-11] [  3.15091161e-10   1.92528592e-10   1.64089591e-10   1.35980100e-10
   1.24735435e-10   1.10946692e-10   1.04587195e-10   9.60467496e-11
   9.18211029e-11   8.58763176e-11   8.28160118e-11]


In [85]:
print((543.1020504*10**(-12)/np.sqrt(nhkl_mess_a2)-543.1020504*10**(-12)/np.sqrt(käse))/(543.1020504*10**(-12)/np.sqrt(käse)))

[  4.88278255e-03   2.67176440e-03   2.06508760e-03   1.50680353e-03
   1.11784568e-03   7.79815145e-04   6.42520733e-04   4.05841635e-04
   2.18964137e-04   5.05821499e-05  -7.57581954e-05]


In [94]:
sigma_a1 = []
gamma_a1 = []
sigma_a2 = []
gamma_a2 = []
for i in range(1,12,1):
    sigma_a1.append(params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i-1)+'sigma'])
    gamma_a1.append(params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i-1)+'gamma'])
    sigma_a2.append(params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i)+'sigma'])
    gamma_a2.append(params_multivoigt_pulver.valuesdict()['voigt_peak_'+str(2*i)+'gamma'])
sigma_a1 = np.array(sigma_a1)
gamma_a1 = np.array(gamma_a1)
sigma_a2 = np.array(sigma_a2)
gamma_a2 = np.array(gamma_a2)
fwhm_a1 = 0.5346*2*gamma_a1+np.sqrt(0.2166*4*gamma_a1**2+8*sigma_a1**2*np.log(2))
fwhm_a2 = 0.5346*2*gamma_a2+np.sqrt(0.2166*4*gamma_a2**2+8*sigma_a2**2*np.log(2))
B_Korn_a1 = 0.94*6.62607004*10**(-34)*299792458/(fwhm_a1*E_a1m_a2m_a1t_a2t[0]*1.6021766208*10**(-19)*np.cos(mess_a1/2*np.pi/180))
B_Korn_a2 = 0.94*6.62607004*10**(-34)*299792458/(fwhm_a2*E_a1m_a2m_a1t_a2t[1]*1.6021766208*10**(-19)*np.cos(mess_a2/2*np.pi/180))
print('B_Korn_a1 = ')
print(B_Korn_a1)
print('B_Korn_a2 = ')
print(B_Korn_a2)
print('fwhm_a1 = ')
print(fwhm_a1)
print('fwhm_a2 = ')
print(fwhm_a2)

B_Korn_a1 = 
[  5.82226669e-09   6.15978795e-09   4.26204276e-09   4.56626227e-09
   5.74023984e-09   7.84013877e-09   4.17068953e-09   4.72158258e-09
   3.76657776e-09   4.24821432e-09   6.12940932e-09]
B_Korn_a2 = 
[  4.67008367e-09   8.23720329e-09   5.13080450e-09   6.87437827e-09
   7.20452617e-09   6.30283702e-09   4.19334213e-09   4.22617524e-09
   4.17903140e-09   6.45380922e-09   4.45985635e-09]
fwhm_a1 = 
[ 0.02563694  0.02563694  0.03845541  0.03845541  0.03204618  0.02563694
  0.05127388  0.05127388  0.07050159  0.07691082  0.06409235]
fwhm_a2 = 
[ 0.03204617  0.01922771  0.03204617  0.02563694  0.02563694  0.03204617
  0.05127388  0.05768312  0.06409235  0.05127388  0.08972929]


In [38]:
plt.errorbar(Silicium_Simulation[0],Silicium_Simulation[1],np.sqrt(Silicium_Simulation[1]),label = r'Simulierte Daten mit Fehler $\sqrt{N}$')
plt.plot(np.arange(Silicium_Simulation[0][0], Silicium_Simulation[0][-1],0.001),fit_multi_voigt_pulverss.eval(x=np.arange(Silicium_Simulation[0][0], Silicium_Simulation[0][-1],0.001)),label = r'Voigtfit')
plt.xticks(size = 24)
plt.yticks(size = 24)
plt.ylim(0,13000)
plt.legend(loc = 'best', fontsize = 24)
plt.ylabel('Counts', size = 24)
plt.xlabel(r'2$\theta/^{\circ}$', size = 24)
plt.show()

In [17]:
Germanium_Simulation = np.loadtxt("Germanium.x_y").T

In [18]:
Germanium_Simulation

array([[  20.   ,   20.01 ,   20.02 , ...,  139.976,  139.986,  139.996],
       [   1.035,    1.037,    1.04 , ...,    0.   ,    0.   ,    0.   ]])

In [19]:
voigt_peak_1gs = models.VoigtModel(prefix='voigt_peak_1')
voigt_peak_2gs = models.VoigtModel(prefix='voigt_peak_2')
voigt_peak_3gs = models.VoigtModel(prefix='voigt_peak_3')
voigt_peak_4gs = models.VoigtModel(prefix='voigt_peak_4')
voigt_peak_5gs = models.VoigtModel(prefix='voigt_peak_5')
voigt_peak_6gs = models.VoigtModel(prefix='voigt_peak_6')
voigt_peak_7gs = models.VoigtModel(prefix='voigt_peak_7')
voigt_peak_8gs = models.VoigtModel(prefix='voigt_peak_8')
voigt_peak_9gs = models.VoigtModel(prefix='voigt_peak_9')
voigt_peak_10gs = models.VoigtModel(prefix='voigt_peak_10')
voigt_peak_11gs = models.VoigtModel(prefix='voigt_peak_11')
voigt_peak_12gs = models.VoigtModel(prefix='voigt_peak_12')
voigt_peak_13gs = models.VoigtModel(prefix='voigt_peak_13')
voigt_peak_14gs = models.VoigtModel(prefix='voigt_peak_14')
voigt_peak_15gs = models.VoigtModel(prefix='voigt_peak_15')
voigt_peak_16gs = models.VoigtModel(prefix='voigt_peak_16')
voigt_peak_17gs = models.VoigtModel(prefix='voigt_peak_17')
voigt_peak_18gs = models.VoigtModel(prefix='voigt_peak_18')
voigt_peak_19gs = models.VoigtModel(prefix='voigt_peak_19')
voigt_peak_20gs = models.VoigtModel(prefix='voigt_peak_20')
voigt_peak_21gs = models.VoigtModel(prefix='voigt_peak_21')
voigt_peak_22gs = models.VoigtModel(prefix='voigt_peak_22')

In [20]:
peak_1_range_pulvergs = np.arange(np.where( 28.24-0.98 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 28.32-0.98)[0][0])
peak_2_range_pulvergs = np.arange(np.where( 28.335-0.98 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 28.4-0.98)[0][0])
peak_3_range_pulvergs = np.arange(np.where( 45.3 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 45.43)[0][0])
peak_4_range_pulvergs = np.arange(np.where( 45.45 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 45.55)[0][0])
peak_5_range_pulvergs = np.arange(np.where( 55.9-2.2 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 56.02-2.2)[0][0])
peak_6_range_pulvergs = np.arange(np.where( 56.06-2.2 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 56.17-2.2)[0][0])
peak_7_range_pulvergs = np.arange(np.where( 66.05 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 66.15)[0][0])
peak_8_range_pulvergs = np.arange(np.where( 66.25 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 66.35)[0][0])
peak_9_range_pulvergs = np.arange(np.where( 72.8 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 73)[0][0])
peak_10_range_pulvergs = np.arange(np.where( 73.1 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 73.2)[0][0])
peak_11_range_pulvergs = np.arange(np.where( 83.7 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 83.9)[0][0])
peak_12_range_pulvergs = np.arange(np.where( 84 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 84.15)[0][0])
peak_13_range_pulvergs = np.arange(np.where( 90.1 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 90.3)[0][0])
peak_14_range_pulvergs = np.arange(np.where( 90.45 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 90.55)[0][0])
peak_15_range_pulvergs = np.arange(np.where( 100.8 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 101)[0][0])
peak_16_range_pulvergs = np.arange(np.where( 101.2 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 101.35)[0][0])
peak_17_range_pulvergs = np.arange(np.where( 107.4 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 107.65)[0][0])
peak_18_range_pulvergs = np.arange(np.where( 107.85 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 108)[0][0])
peak_19_range_pulvergs = np.arange(np.where( 119 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 119.25)[0][0])
peak_20_range_pulvergs = np.arange(np.where( 119.55 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 119.8)[0][0])
peak_21_range_pulvergs = np.arange(np.where( 126.65 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 126.85)[0][0])
peak_22_range_pulvergs = np.arange(np.where( 127.2 < Germanium_Simulation[0])[0][0],np.where( Germanium_Simulation[0] > 127.45)[0][0])

In [21]:
params_peak_1gs = voigt_peak_1gs.guess(Germanium_Simulation[1][peak_1_range_pulvergs],x=Germanium_Simulation[0][peak_1_range_pulvergs])
params_peak_2gs = voigt_peak_2gs.guess(Germanium_Simulation[1][peak_2_range_pulvergs],x=Germanium_Simulation[0][peak_2_range_pulvergs])
params_peak_3gs = voigt_peak_3gs.guess(Germanium_Simulation[1][peak_3_range_pulvergs],x=Germanium_Simulation[0][peak_3_range_pulvergs])
params_peak_4gs = voigt_peak_4gs.guess(Germanium_Simulation[1][peak_4_range_pulvergs],x=Germanium_Simulation[0][peak_4_range_pulvergs])
params_peak_5gs = voigt_peak_5gs.guess(Germanium_Simulation[1][peak_5_range_pulvergs],x=Germanium_Simulation[0][peak_5_range_pulvergs])
params_peak_6gs = voigt_peak_6gs.guess(Germanium_Simulation[1][peak_6_range_pulvergs],x=Germanium_Simulation[0][peak_6_range_pulvergs])
params_peak_7gs = voigt_peak_7gs.guess(Germanium_Simulation[1][peak_7_range_pulvergs],x=Germanium_Simulation[0][peak_7_range_pulvergs])
params_peak_8gs = voigt_peak_8gs.guess(Germanium_Simulation[1][peak_8_range_pulvergs],x=Germanium_Simulation[0][peak_8_range_pulvergs])
params_peak_9gs = voigt_peak_9gs.guess(Germanium_Simulation[1][peak_9_range_pulvergs],x=Germanium_Simulation[0][peak_9_range_pulvergs])
params_peak_10gs = voigt_peak_10gs.guess(Germanium_Simulation[1][peak_10_range_pulvergs],x=Germanium_Simulation[0][peak_10_range_pulvergs])
params_peak_11gs = voigt_peak_11gs.guess(Germanium_Simulation[1][peak_11_range_pulvergs],x=Germanium_Simulation[0][peak_11_range_pulvergs])
params_peak_12gs = voigt_peak_12gs.guess(Germanium_Simulation[1][peak_12_range_pulvergs],x=Germanium_Simulation[0][peak_12_range_pulvergs])
params_peak_13gs = voigt_peak_13gs.guess(Germanium_Simulation[1][peak_13_range_pulvergs],x=Germanium_Simulation[0][peak_13_range_pulvergs])
params_peak_14gs = voigt_peak_14gs.guess(Germanium_Simulation[1][peak_14_range_pulvergs],x=Germanium_Simulation[0][peak_14_range_pulvergs])
params_peak_15gs = voigt_peak_15gs.guess(Germanium_Simulation[1][peak_15_range_pulvergs],x=Germanium_Simulation[0][peak_15_range_pulvergs])
params_peak_16gs = voigt_peak_16gs.guess(Germanium_Simulation[1][peak_16_range_pulvergs],x=Germanium_Simulation[0][peak_16_range_pulvergs])
params_peak_17gs = voigt_peak_17gs.guess(Germanium_Simulation[1][peak_17_range_pulvergs],x=Germanium_Simulation[0][peak_17_range_pulvergs])
params_peak_18gs = voigt_peak_18gs.guess(Germanium_Simulation[1][peak_18_range_pulvergs],x=Germanium_Simulation[0][peak_18_range_pulvergs])
params_peak_19gs = voigt_peak_19gs.guess(Germanium_Simulation[1][peak_19_range_pulvergs],x=Germanium_Simulation[0][peak_19_range_pulvergs])
params_peak_20gs = voigt_peak_20gs.guess(Germanium_Simulation[1][peak_20_range_pulvergs],x=Germanium_Simulation[0][peak_20_range_pulvergs])
params_peak_21gs = voigt_peak_21gs.guess(Germanium_Simulation[1][peak_21_range_pulvergs],x=Germanium_Simulation[0][peak_21_range_pulvergs])
params_peak_22gs = voigt_peak_22gs.guess(Germanium_Simulation[1][peak_22_range_pulvergs],x=Germanium_Simulation[0][peak_22_range_pulvergs])

In [22]:
params_peak_1gs['voigt_peak_1gamma'].set(value = params_peak_1gs.valuesdict()['voigt_peak_1sigma']/200 ,vary=True, expr='')
params_peak_2gs['voigt_peak_2gamma'].set(value = params_peak_2gs.valuesdict()['voigt_peak_2sigma']/200 ,vary=True, expr='')
params_peak_3gs['voigt_peak_3gamma'].set(value = params_peak_3gs.valuesdict()['voigt_peak_3sigma']/200 ,vary=True, expr='')
params_peak_4gs['voigt_peak_4gamma'].set(value = params_peak_4gs.valuesdict()['voigt_peak_4sigma']/200 ,vary=True, expr='')
params_peak_5gs['voigt_peak_5gamma'].set(value = params_peak_5gs.valuesdict()['voigt_peak_5sigma']/200 ,vary=True, expr='')
params_peak_6gs['voigt_peak_6gamma'].set(value = params_peak_6gs.valuesdict()['voigt_peak_6sigma']/200 ,vary=True, expr='')
params_peak_7gs['voigt_peak_7gamma'].set(value = params_peak_7gs.valuesdict()['voigt_peak_7sigma']/200 ,vary=True, expr='')
params_peak_8gs['voigt_peak_8gamma'].set(value = params_peak_8gs.valuesdict()['voigt_peak_8sigma']/200 ,vary=True, expr='')
params_peak_9gs['voigt_peak_9gamma'].set(value = params_peak_9gs.valuesdict()['voigt_peak_9sigma']/200 ,vary=True, expr='')
params_peak_10gs['voigt_peak_10gamma'].set(value = params_peak_10gs.valuesdict()['voigt_peak_10sigma']/200 ,vary=True, expr='')
params_peak_11gs['voigt_peak_11gamma'].set(value = params_peak_11gs.valuesdict()['voigt_peak_11sigma']/200 ,vary=True, expr='')
params_peak_12gs['voigt_peak_12gamma'].set(value = params_peak_12gs.valuesdict()['voigt_peak_12sigma']/200 ,vary=True, expr='')
params_peak_13gs['voigt_peak_13gamma'].set(value = params_peak_13gs.valuesdict()['voigt_peak_13sigma']/200 ,vary=True, expr='')
params_peak_14gs['voigt_peak_14gamma'].set(value = params_peak_14gs.valuesdict()['voigt_peak_14sigma']/200 ,vary=True, expr='')
params_peak_15gs['voigt_peak_15gamma'].set(value = params_peak_15gs.valuesdict()['voigt_peak_15sigma']/200 ,vary=True, expr='')
params_peak_16gs['voigt_peak_16gamma'].set(value = params_peak_16gs.valuesdict()['voigt_peak_16sigma']/200 ,vary=True, expr='')
params_peak_17gs['voigt_peak_17gamma'].set(value = params_peak_17gs.valuesdict()['voigt_peak_17sigma']/200 ,vary=True, expr='')
params_peak_18gs['voigt_peak_18gamma'].set(value = params_peak_18gs.valuesdict()['voigt_peak_18sigma']/200 ,vary=True, expr='')
params_peak_19gs['voigt_peak_19gamma'].set(value = params_peak_19gs.valuesdict()['voigt_peak_19sigma']/200 ,vary=True, expr='')
params_peak_20gs['voigt_peak_20gamma'].set(value = params_peak_20gs.valuesdict()['voigt_peak_20sigma']/200 ,vary=True, expr='')
params_peak_21gs['voigt_peak_21gamma'].set(value = params_peak_21gs.valuesdict()['voigt_peak_21sigma']/200 ,vary=True, expr='')
params_peak_22gs['voigt_peak_22gamma'].set(value = params_peak_22gs.valuesdict()['voigt_peak_22sigma']/200 ,vary=True, expr='')

In [23]:
#ab hier wiederholschleife
params_peak_1gs['voigt_peak_sigma'].set(value = params_peak_1gs.valuesdict()['voigt_peak_1sigma'] ,vary=True, expr='')
params_peak_2gs['voigt_peak_2sigma'].set(value = params_peak_2gs.valuesdict()['voigt_peak_2sigma'] ,vary=True, expr='')
params_peak_3gs['voigt_peak_3sigma'].set(value = params_peak_3gs.valuesdict()['voigt_peak_3sigma'] ,vary=True, expr='')
params_peak_4gs['voigt_peak_4sigma'].set(value = params_peak_4gs.valuesdict()['voigt_peak_4sigma'] ,vary=True, expr='')
params_peak_5gs['voigt_peak_5sigma'].set(value = params_peak_5gs.valuesdict()['voigt_peak_5sigma'] ,vary=True, expr='')
params_peak_6gs['voigt_peak_6sigma'].set(value = params_peak_6gs.valuesdict()['voigt_peak_6sigma'] ,vary=True, expr='')
params_peak_7gs['voigt_peak_7sigma'].set(value = params_peak_7gs.valuesdict()['voigt_peak_7sigma'] ,vary=True, expr='')
params_peak_8gs['voigt_peak_8sigma'].set(value = params_peak_8gs.valuesdict()['voigt_peak_8sigma'] ,vary=True, expr='')
params_peak_9gs['voigt_peak_9sigma'].set(value = params_peak_9gs.valuesdict()['voigt_peak_9sigma'] ,vary=True, expr='')
params_peak_10gs['voigt_peak_10sigma'].set(value = params_peak_10gs.valuesdict()['voigt_peak_10sigma'] ,vary=True, expr='')
params_peak_11gs['voigt_peak_11sigma'].set(value = params_peak_11gs.valuesdict()['voigt_peak_11sigma'] ,vary=True, expr='')
params_peak_12gs['voigt_peak_12sigma'].set(value = params_peak_12gs.valuesdict()['voigt_peak_12sigma'] ,vary=True, expr='')
params_peak_13gs['voigt_peak_13sigma'].set(value = params_peak_13gs.valuesdict()['voigt_peak_13sigma'] ,vary=True, expr='')
params_peak_14gs['voigt_peak_14sigma'].set(value = params_peak_14gs.valuesdict()['voigt_peak_14sigma'] ,vary=True, expr='')
params_peak_15gs['voigt_peak_15sigma'].set(value = params_peak_15gs.valuesdict()['voigt_peak_15sigma'] ,vary=True, expr='')
params_peak_16gs['voigt_peak_16sigma'].set(value = params_peak_16gs.valuesdict()['voigt_peak_16sigma'] ,vary=True, expr='')
params_peak_17gs['voigt_peak_17sigma'].set(value = params_peak_17gs.valuesdict()['voigt_peak_17sigma'] ,vary=True, expr='')
params_peak_18gs['voigt_peak_18sigma'].set(value = params_peak_18gs.valuesdict()['voigt_peak_18sigma'] ,vary=True, expr='')
params_peak_19gs['voigt_peak_19sigma'].set(value = params_peak_19gs.valuesdict()['voigt_peak_19sigma'] ,vary=True, expr='')
params_peak_20gs['voigt_peak_20sigma'].set(value = params_peak_20gs.valuesdict()['voigt_peak_20sigma'] ,vary=True, expr='')
params_peak_21gs['voigt_peak_21sigma'].set(value = params_peak_21gs.valuesdict()['voigt_peak_21sigma'] ,vary=True, expr='')
params_peak_22gs['voigt_peak_22sigma'].set(value = params_peak_22gs.valuesdict()['voigt_peak_22sigma'] ,vary=True, expr='')

params_peak_1gs['voigt_peak_1gamma'].set(value = params_peak_1gs.valuesdict()['voigt_peak_1gamma'] ,vary=True, expr='')
params_peak_2gs['voigt_peak_2gamma'].set(value = params_peak_2gs.valuesdict()['voigt_peak_2gamma'] ,vary=True, expr='')
params_peak_3gs['voigt_peak_3gamma'].set(value = params_peak_3gs.valuesdict()['voigt_peak_3gamma'] ,vary=True, expr='')
params_peak_4gs['voigt_peak_4gamma'].set(value = params_peak_4gs.valuesdict()['voigt_peak_4gamma'] ,vary=True, expr='')
params_peak_5gs['voigt_peak_5gamma'].set(value = params_peak_5gs.valuesdict()['voigt_peak_5gamma'] ,vary=True, expr='')
params_peak_6gs['voigt_peak_6gamma'].set(value = params_peak_6gs.valuesdict()['voigt_peak_6gamma'] ,vary=True, expr='')
params_peak_7gs['voigt_peak_7gamma'].set(value = params_peak_7gs.valuesdict()['voigt_peak_7gamma'] ,vary=True, expr='')
params_peak_8gs['voigt_peak_8gamma'].set(value = params_peak_8gs.valuesdict()['voigt_peak_8gamma'] ,vary=True, expr='')
params_peak_9gs['voigt_peak_9gamma'].set(value = params_peak_9gs.valuesdict()['voigt_peak_9gamma'] ,vary=True, expr='')
params_peak_10gs['voigt_peak_10gamma'].set(value = params_peak_10gs.valuesdict()['voigt_peak_10gamma'] ,vary=True, expr='')
params_peak_11gs['voigt_peak_11gamma'].set(value = params_peak_11gs.valuesdict()['voigt_peak_11gamma'] ,vary=True, expr='')
params_peak_12gs['voigt_peak_12gamma'].set(value = params_peak_12gs.valuesdict()['voigt_peak_12gamma'] ,vary=True, expr='')
params_peak_13gs['voigt_peak_13gamma'].set(value = params_peak_13gs.valuesdict()['voigt_peak_13gamma'] ,vary=True, expr='')
params_peak_14gs['voigt_peak_14gamma'].set(value = params_peak_14gs.valuesdict()['voigt_peak_14gamma'] ,vary=True, expr='')
params_peak_15gs['voigt_peak_15gamma'].set(value = params_peak_15gs.valuesdict()['voigt_peak_15gamma'] ,vary=True, expr='')
params_peak_16gs['voigt_peak_16gamma'].set(value = params_peak_16gs.valuesdict()['voigt_peak_16gamma'] ,vary=True, expr='')
params_peak_17gs['voigt_peak_17gamma'].set(value = params_peak_17gs.valuesdict()['voigt_peak_17gamma'] ,vary=True, expr='')
params_peak_18gs['voigt_peak_18gamma'].set(value = params_peak_18gs.valuesdict()['voigt_peak_18gamma'] ,vary=True, expr='')
params_peak_19gs['voigt_peak_19gamma'].set(value = params_peak_19gs.valuesdict()['voigt_peak_19gamma'] ,vary=True, expr='')
params_peak_20gs['voigt_peak_20gamma'].set(value = params_peak_20gs.valuesdict()['voigt_peak_20gamma'] ,vary=True, expr='')
params_peak_21gs['voigt_peak_21gamma'].set(value = params_peak_21gs.valuesdict()['voigt_peak_21gamma'] ,vary=True, expr='')
params_peak_22gs['voigt_peak_22gamma'].set(value = params_peak_22gs.valuesdict()['voigt_peak_22gamma'] ,vary=True, expr='')

In [24]:
multi_voigt_pulvergs = voigt_peak_1gs + voigt_peak_2gs + voigt_peak_3gs + voigt_peak_4gs + voigt_peak_5gs + voigt_peak_6gs + voigt_peak_7gs + voigt_peak_8gs + voigt_peak_9gs + voigt_peak_10gs + voigt_peak_11gs + voigt_peak_12gs + voigt_peak_13gs + voigt_peak_14gs + voigt_peak_15gs + voigt_peak_16gs + voigt_peak_17gs + voigt_peak_18gs + voigt_peak_19gs + voigt_peak_20gs + voigt_peak_21gs + voigt_peak_22gs 
params_multivoigt_pulvergs = multi_voigt_pulvergs.make_params()
params_multivoigt_pulvergs.update(params_peak_1gs)
params_multivoigt_pulvergs.update(params_peak_2gs)
params_multivoigt_pulvergs.update(params_peak_3gs)
params_multivoigt_pulvergs.update(params_peak_4gs)
params_multivoigt_pulvergs.update(params_peak_5gs)
params_multivoigt_pulvergs.update(params_peak_6gs)
params_multivoigt_pulvergs.update(params_peak_7gs)
params_multivoigt_pulvergs.update(params_peak_8gs)
params_multivoigt_pulvergs.update(params_peak_9gs)
params_multivoigt_pulvergs.update(params_peak_10gs)
params_multivoigt_pulvergs.update(params_peak_11gs)
params_multivoigt_pulvergs.update(params_peak_12gs)
params_multivoigt_pulvergs.update(params_peak_13gs)
params_multivoigt_pulvergs.update(params_peak_14gs)
params_multivoigt_pulvergs.update(params_peak_15gs)
params_multivoigt_pulvergs.update(params_peak_16gs)
params_multivoigt_pulvergs.update(params_peak_17gs)
params_multivoigt_pulvergs.update(params_peak_18gs)
params_multivoigt_pulvergs.update(params_peak_19gs)
params_multivoigt_pulvergs.update(params_peak_20gs)
params_multivoigt_pulvergs.update(params_peak_21gs)
params_multivoigt_pulvergs.update(params_peak_22gs)

In [25]:
fit_multi_voigt_pulvergs = multi_voigt_pulvergs.fit(Germanium_Simulation[1][np.where(Germanium_Simulation[1] > -1)],params_multivoigt_pulvergs,x=Germanium_Simulation[0][np.where(Germanium_Simulation[1] > -1)])
print(fit_multi_voigt_pulvergs.fit_report())

[[Model]]
    (((((((((((((((((((((Model(voigt, prefix='voigt_peak_1') + Model(voigt, prefix='voigt_peak_2')) + Model(voigt, prefix='voigt_peak_3')) + Model(voigt, prefix='voigt_peak_4')) + Model(voigt, prefix='voigt_peak_5')) + Model(voigt, prefix='voigt_peak_6')) + Model(voigt, prefix='voigt_peak_7')) + Model(voigt, prefix='voigt_peak_8')) + Model(voigt, prefix='voigt_peak_9')) + Model(voigt, prefix='voigt_peak_10')) + Model(voigt, prefix='voigt_peak_11')) + Model(voigt, prefix='voigt_peak_12')) + Model(voigt, prefix='voigt_peak_13')) + Model(voigt, prefix='voigt_peak_14')) + Model(voigt, prefix='voigt_peak_15')) + Model(voigt, prefix='voigt_peak_16')) + Model(voigt, prefix='voigt_peak_17')) + Model(voigt, prefix='voigt_peak_18')) + Model(voigt, prefix='voigt_peak_19')) + Model(voigt, prefix='voigt_peak_20')) + Model(voigt, prefix='voigt_peak_21')) + Model(voigt, prefix='voigt_peak_22'))
[[Fit Statistics]]
    # function evals   = 1696
    # data points      = 11979
    # variables  

In [56]:
plt.errorbar(Germanium_Simulation[0],Germanium_Simulation[1],np.sqrt(Germanium_Simulation[1]),label = r'Simulierte Daten mit Fehler $\sqrt{N}$')
plt.plot(np.arange(Germanium_Simulation[0][0], Germanium_Simulation[0][-1],0.001),fit_multi_voigt_pulvergs.eval(x=np.arange(Germanium_Simulation[0][0], Germanium_Simulation[0][-1],0.001)),label = r'Voigtfit')
plt.xticks(size = 24)
plt.yticks(size = 24)
plt.ylim(0,70000)
plt.legend(loc = 'best', fontsize = 24)
plt.ylabel('Counts', size = 24)
plt.xlabel(r'2$\theta/^{\circ}$', size = 24)
plt.show()